### Testing out different apis with different inputs

Read in publications.json and identify a random entry

In [26]:
import json
with open('all_pubs.json') as json_file:
    publications = json.load(json_file)

#### Fetch by title

##### Dimensions

In [34]:
import RichContextAPI
import importlib
importlib.reload(RichContextAPI)

<module 'RichContextAPI' from '/Users/sophierand/RCApi/apis/RichContextAPI.py'>

In [28]:
test_case = publications[0]
# test_case = publications[44]

In [35]:
dimensions_results = RichContextAPI.dimensions_title_search(test_case['title'])

API credentials have been set


In [38]:
# dimensions_results['title']

##### SSRN

In [22]:
test_case = publications[44]

In [23]:
ssrn_results = RichContextAPI.ssrn_main(test_case)

##### RePEc

In [ ]:
import urllib
from bs4 import BeautifulSoup

In [ ]:
def get_repec_handle (title):
    enc_title = urllib.parse.quote_plus(title.replace("(", "").replace(")", "").replace(":", ""))

    cgi_url = "https://ideas.repec.org/cgi-bin/htsearch?q={}".format(enc_title)
    response = requests.get(cgi_url).text
    #print(BeautifulSoup(response, "html.parser").prettify())

    soup = BeautifulSoup(response, "html.parser")
    ol = soup.find("ol", {"class": "list-group"})
    results = ol.findChildren()

    if len(results) > 0:
        li = results[0]
        handle = li.find("i").get_text()
        return handle
    else:
        return None

In [ ]:
title = "The use of cash and debit cards as payment instruments in Colombia"

In [ ]:
a = get_repec_handle(title)

In [ ]:
a

In [ ]:
import requests
# handle = "RePEc:lde:journl:y:2019:i:90:p:71-95"
token = 'd9kHt3Cm'

In [ ]:
api_url = "https://api.repec.org/call.cgi?code={}&getref={}".format(token, a)
response = requests.get(api_url).text

##### EuropePMC

In [ ]:
def get_europepmc_metadata (url):
    """
    parse metadata from a Europe PMC web page for a publication
    """

    # <meta content="http://europepmc.org/articles/PMC2819787?pdf=render" name="citation_pdf_url"/>
    # <meta content="10.3390/ijerph7010269" name="citation_doi"/>
    # <meta content="Int J Environ Res Public Health" name="citation_journal_abbrev"/>

    response = requests.get(url).text
    #print(BeautifulSoup(response, "html.parser").prettify())

    publisher = None
    doi = None
    pdf = None

    soup = BeautifulSoup(response, "html.parser")

    for x in soup.find_all("span", {"id": "pmcmata"}):
        publisher = x.get_text()

    for x in soup.find_all("meta",  {"name": "citation_doi"}):
        doi = x["content"]

    for x in soup.find_all("meta",  {"name": "citation_pdf_url"}):
        pdf = x["content"]

    if publisher and doi and pdf:
        return [publisher, doi, pdf]
    else:
        return None


### MAIN

In [ ]:
if 'ssrn' in test_case['doi'] or 'ssrn' in test_case['url']:
    ssrn_results = RichContextAPI.ssrn_main(test_case)